### Heading into the forest - The National Forest!
<img src ='images/sam-frodo-gollum.alienationmentale.wordpress.com.jpg'  style="display: inline-block" width=60%>
<br>
image source: alienationmentale.wordpress.com
<br>
<br>
#### Scraping techniques presented here can be used on pages like:
http://www.fs.usda.gov/recarea/mthood/recarea/?recid=53228

In [1]:
import requests	
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
from sqlalchemy import create_engine
import config
from unidecode import unidecode
import datetime

<img src="images/lady_galadriel.www.xenocorp.net.jpg" width=20%  style="display: inline-block">
<br>
image source: www.xenocorp.net

#### USFS site urls, a gift from the elves


In [2]:
fs_urls = pd.read_csv('data/usfs_sites.csv')

In [3]:
fs_urls

,facilityname,url
0,whispering falls campground,Willamette National Forest - Whispering Falls ...
1,lost lake campground,Mt. Hood National Forest - Lost Lake Campgroun...
2,lake harriet campground,Mt. Hood National Forest - Lake Harriet Campgr...
3,kinnikinnick (laurance lake) campground,Mt. Hood National Forest - Kinnikinnick (Laura...
4,fifteenmile campground,Mt. Hood National Forest - Fifteenmile Campgro...
5,clear lake campground,Mt. Hood National Forest - Clear Lake Campgrou...
6,camp creek campground,Mt. Hood National Forest - Camp Creek Campgrou...
7,badger lake campground,Mt. Hood National Forest - Badger Lake Campgro...
8,eagle creek campground,Columbia River Gorge National Scenic Area - Ea...


In [4]:
cg_name = fs_urls.iloc[1]['facilityname']

Connect to LAMP container to retrieve Forest Service website

In [5]:
site_url = "http://" + config.LAMP_IP + "/" + fs_urls.iloc[1]['url']
cg_req = requests.get(site_url)
cg_soup = BeautifulSoup(cg_req.text, 'lxml')

If above does not work for you, read the file locally:

In [6]:
site_url = fs_urls.iloc[1]['url']
cg_data = open("webfiles/" + site_url,'r').read()
cg_soup = BeautifulSoup(cg_data, 'lxml')

Extracting Area Status information

In [7]:
try :
    for strong_tag in cg_soup.find_all('strong'):
        if ('Area Status' in unidecode(strong_tag.text)):
            status = unidecode(strong_tag.next_sibling).strip()
except Exception:
    print('couldnt get area status')

In [8]:
status

'Open'

Extracting lat/long information

In [9]:
try :
    lat = cg_soup.find_all('div', text=re.compile('Latitude'))
    div = [row.next_sibling.next_sibling for row in lat]
    latitude  = div[0].text.strip()

except Exception:
    print('couldnt get location info')

In [10]:
latitude

'45.50080'

In [11]:
def get_location(soup, search_text):
    try :
        divs = soup.find_all('div', text=re.compile(search_text))
        loc_div = [row.next_sibling.next_sibling for row in divs]
        return loc_div[0].text.strip()
    except Exception as ex:
        print("get_location: couldnt extract " + search_text)
        return ""

In [12]:
cg_lat = get_location(cg_soup, 'Latitude')
cg_long = get_location(cg_soup, 'Longitude')
cg_elev = get_location(cg_soup, 'Elevation')

In [13]:
print(str(cg_lat) + "," + str(cg_long) + " elevation:" + str(cg_elev))

45.50080,-121.81641 elevation:3200


Extracting campground info

In [14]:
try :
    tables = cg_soup.find_all('div', {'class': 'tablecolor'})
except Exception:
    print('couldnt get tables')

In [15]:
try :
    print(len(tables))
    rows = tables[0].find_all('tr')
    for row in rows:
        if row.th.text == 'Reservations:':
            reservations = unidecode(row.td.text).strip()
        if row.th.text == 'Open Season:':
            openseason = unidecode(row.td.text).strip()
        if row.th.text == 'Current Conditions:':
            conditions = unidecode(row.td.text).strip()
        if row.th.text == 'Water:':
            water = unidecode(row.td.text).strip()
        if row.th.text == 'Restroom:':
            restroom = unidecode(row.td.text).strip()
except Exception as ex:
    print('couldnt get basic campground info')
    print(ex)
    
df_info = pd.DataFrame({
        'FacilityName':[cg_name],
        'Reservations':[reservations],
        'OpenSeason':[openseason],
        'CurrentConditions':[conditions],
        'Water':[water],
        'Restroom':[restroom]
    })

10


In [16]:
df_info

,CurrentConditions,FacilityName,OpenSeason,Reservations,Restroom,Water
0,OPEN,lost lake campground,April,This site can be reserved by calling Toll Free...,Vault Toilet (18),Potable Water
